In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
data.head()
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
G = len(np.unique(y))
X = np.delete(X,[0,1], axis = 1)
for g in range(G):
  print(sum(y==g))
X.shape

126
225


(351, 32)

In [3]:
missing_rates = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([100,-80])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["0%", "10%", "20%", "30%", "40%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM          0.82$\pm$0.054  0.939$\pm$0.017   
                         LR          0.808$\pm$0.032  0.834$\pm$0.028   
                         DCT         0.791$\pm$0.063  0.866$\pm$0.033   
             sensitivity SVM         0.817$\pm$0.055   0.94$\pm$0.017   
                         LR          0.811$\pm$0.033  0.837$\pm$0.027   
                         DCT         0.789$\pm$0.064  0.866$\pm$0.032   
10%          F-1 score   SVM         0.787$\pm$0.052  0.942$\pm$0.007   
                         LR          0.791$\pm$0.062  0.851$\pm$0.016   
                         DCT         0.789$\pm$0.046   0.87$\pm$0.022   
             sensitivity SVM         0.784$\pm$0.051  0.943$\pm$0.007   
                         LR          0.794$\pm$0.064  0.854$\pm$0.014   
                         DCT         0.786$\pm$0.047   0.87$\pm$0.022   
20%          F-1 score   SVM         0.758$\pm$0.036  0.936$\pm$0.016   
                         LR           0.784$\pm$0.04  0.833$\pm$0.026   
                         DCT         0.737$\pm$0.049  0.845$\pm$0.032   
             sensitivity SVM         0.755$\pm$0.036  0.936$\pm$0.016   
                         LR          0.786$\pm$0.042  0.835$\pm$0.025   
                         DCT          0.733$\pm$0.05  0.845$\pm$0.032   
30%          F-1 score   SVM         0.745$\pm$0.027   0.94$\pm$0.018   
                         LR          0.757$\pm$0.053  0.817$\pm$0.021   
                         DCT         0.746$\pm$0.061  0.857$\pm$0.035   
             sensitivity SVM          0.74$\pm$0.028   0.94$\pm$0.018   
                         LR          0.758$\pm$0.057    0.82$\pm$0.02   
                         DCT         0.741$\pm$0.063  0.857$\pm$0.035   
40%          F-1 score   SVM         0.754$\pm$0.041   0.94$\pm$0.013   
                         LR          0.735$\pm$0.068  0.807$\pm$0.025   
                         DCT         0.745$\pm$0.049  0.843$\pm$0.042   
             sensitivity SVM          0.75$\pm$0.041   0.94$\pm$0.013   
                         LR          0.733$\pm$0.072  0.809$\pm$0.025   
                         DCT          0.74$\pm$0.049  0.842$\pm$0.042   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.934$\pm$0.016   0.94$\pm$0.019   
                         LR          0.819$\pm$0.023  0.827$\pm$0.024   
                         DCT         0.864$\pm$0.032  0.862$\pm$0.023   
             sensitivity SVM         0.935$\pm$0.016   0.94$\pm$0.019   
                         LR          0.823$\pm$0.022  0.828$\pm$0.023   
                         DCT         0.866$\pm$0.032  0.862$\pm$0.023   
10%          F-1 score   SVM         0.941$\pm$0.014   0.95$\pm$0.012   
                         LR          0.847$\pm$0.019  0.843$\pm$0.022   
                         DCT         0.872$\pm$0.017  0.865$\pm$0.037   
             sensitivity SVM         0.941$\pm$0.014   0.95$\pm$0.012   
                         LR          0.852$\pm$0.019  0.845$\pm$0.022   
                         DCT         0.873$\pm$0.017  0.865$\pm$0.038   
20%          F-1 score   SVM         0.935$\pm$0.019  0.941$\pm$0.015   
                         LR          0.832$\pm$0.034  0.824$\pm$0.026   
                         DCT         0.833$\pm$0.028  0.831$\pm$0.026   
             sensitivity SVM         0.935$\pm$0.019  0.941$\pm$0.015   
                         LR          0.835$\pm$0.033  0.826$\pm$0.026   
                         DCT         0.834$\pm$0.027  0.831$\pm$0.026   
30%          F-1 score   SVM          0.929$\pm$0.02  0.941$\pm$0.019   
                         LR          0.819$\pm$0.022  0.806$\pm$0.023   
                         DCT         0.835$\pm$0.016  0.867$\pm$0.042   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_Inosphere_low.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
0\% & F-1 score & SVM &   0.82\$\textbackslash pm\$0.054 &  0.939\$\textbackslash pm\$0.017 &  0.934\$\textbackslash pm\$0.016 &   0.94\$\textbackslash pm\$0.019 &  0.942\$\textbackslash pm\$0.022 &  0.942\$\textbackslash pm\$0.017 \\
    &             & LR &  0.808\$\textbackslash pm\$0.032 &  0.834\$\textbackslash pm\$0.028 &  0.819\$\textbackslash pm\$0.023 &  0.827\$\textbackslash pm\$0.024 &  0.828\$\textbackslash pm\$0.023 &  0.825\$\textbackslash pm\$0.027 \\
    &             & DCT &  0.791\$\textbackslash pm\$0.063 &  0.866\$\textbackslash pm\$0.033 &  0.864\$\textbackslash pm\$0.032 &  0.862\$\textbackslash pm\$0.023 &  0.849\$\textbackslash pm